In [53]:
import sys
sys.path.insert(1, '/Users/mac/Desktop/PycharmProjects/TAADL/src')
sys.path.insert(2, '/Users/mac/Desktop/PycharmProjects/TAADL/models')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from scipy import stats
from config import DATA_PATH
from typing import Optional, Tuple
from scipy.stats import norm
from statsmodels.distributions.empirical_distribution import ECDF
from torch.distributions.multivariate_normal import MultivariateNormal

In [244]:
from scipy.interpolate import interp1d

class ECDF_(object):
    def __init__(self, data:np.ndarray):
        self.x = data
        self.x.sort()
        self.x_min, self.x_max = self.x[0], self.x[-1]
        self.n = len(self.x)
        self.y = np.linspace(1.0/self.n, 1.0, self.n)
        self.f = interp1d(self.x, self.y, fill_value='extrapolate') # make interpolation
        self.inv_f = interp1d(self.y, self.x, fill_value='extrapolate') # inverse is just arguments reversed
        
    def __call__(self, x:np.ndarray) -> np.ndarray:
        """
        calculates y given x under defined ECDF class.
        """
        if np.sum(x > self.x_max) > 0 or np.sum(x < self.x_min) > 0:
            x = np.where(x > self.x_max, self.x_max, x)
            x = np.where(x < self.x_min, self.x_min, x)
        return self.f(x)

    def inverse(self, y:np.ndarray) -> np.ndarray:
        """
        calculates the inverse of ECDF with y as an input.
        """
         # if cdf value is less than 0 or more than 1, trim values
        if np.sum(y > 1.0) > 0 or np.sum(y < 0.0) > 0:
            y = np.where(y > 1.0, 1.0, y)
            y = np.where(y < 0.0, 0.0, y)
        # otherwise, return
        return self.inv_f(y)

def gaussian_loss(x:torch.Tensor, mu_t:torch.Tensor, cov_t:torch.Tensor) -> torch.Tensor:
    assert x.size(0) == mu_t.size(0), \
        'sequence length is not equal (input and mu_t)'
    assert mu_t.size(0) == cov_t.size(0), \
        'sequence length is not equal (mu_t and cov_t)'    
    assert len(cov_t.size()) == 3, \
        'dimension of covariance matrix is not equal to 3.'

    if len(mu_t.size()) == 3:
        mu_t = mu_t.squeeze(2)

    # set a mul;tivariate distribution indexed by time t
    # loc: batch_size x dim, cov: batch_size x dim x dim
    # input x: batch_size x dim
    distribution_t = MultivariateNormal(mu_t, cov_t)
    loglikelihood = distribution_t.log_prob(x)
    return -loglikelihood.sum() # negative log-likelihood


def transform(Z:torch.Tensor, cdfs:Optional[dict]=None) -> Tuple[torch.Tensor,dict]:
    """
    transforms data distribution to standard normal distribution. 
    
    Transform takes 2 steps:
    1. estimate empirical distribution of the data per asset. Then convert it to uniform, u [0,1] distribution
        note that we use step function based empirical CDF which is different to the one specified in the original paper.
        ** values are truncated to prevent standard normal variable goes either -inf or inf.

    2. use inverse CDF of standard normal to convert u to x, where x follows standard normal distribution.
    
      (1)  (2)
    z -> u -> x 

    INPUTS
        df: pd.DataFrame
            input data sequence, that is multivariate
        context_len: Optional[int]
            specifies context length for the training set. rest of the data will be used for prediction.
    
    RETURNS
        X: pd.DataFrame
            returns transformed dataset.
    """
    
    X = []
    m = Z.size(0)
    emp_distributions = {}
    
    # lower and upper bound for emp. CDF
    delta_m = (4 * np.sqrt(np.log(m) * np.pi) * m ** 0.25) ** -1 

    for i in range(Z.size(1)):
        Z_i = Z[:,i].numpy()
        
        # estimate empirical CDF
        # only use m datapoint to estimate CDF.
        if cdfs is not None:
            emp_cdf = cdfs['CDF_'+str(i)]
        else:
            emp_cdf = ECDF_(Z_i.reshape(-1))
        
        # for each datapoint, transform
        U_i = emp_cdf(Z_i)
        
        # truncate extreme values
        U_i = np.where(U_i < delta_m, delta_m, U_i) 
        U_i = np.where(U_i > 1-delta_m, 1-delta_m, U_i)
        
        # get standard normal values
        X_i = norm.ppf(q=U_i, loc=0, scale=1) # inverse CDF of standard normal
        X.append(torch.Tensor(X_i))
        emp_distributions['CDF_'+str(i)] = emp_cdf
        
    # make a dataframe
    X = torch.stack(X, axis=0).T
    return X, emp_distributions


def inv_transform(X:torch.Tensor, cdfs:dict) -> pd.DataFrame:
    """
    inverse transforms standard normal distribution to the original distribution given a set of
    empirical cdfs.
    
    Transform takes 2 steps:
    1. use standard normal to convert x to u, where u is in [0,1].
    2. the input cdfs contains empirical cdfs with its inverse. Using inverse of these cdfs, we transform u to z.
    
      (1)  (2)
    x -> u -> z

    INPUTS
        X: torch.Tensor
            input data sequence, that is multivariate
        cdfs: dict
            a dictionary of empirical cdfs indexed by asset number.
    
    RETURNS
        Z: torch.Tensor
            returns original dataset.
    """ 
    Z = []

    for i in range(X.size(1)):
        X_i = X[:,i].numpy()
        
        # for each datapoint, transform to u by applying standard normal CDF
        U_i = norm.cdf(X_i)
        
        # get empirical distribution of each asset.
        emp_cdf = cdfs['CDF_'+str(i)]

        # transform the uniform data to Z by inverse empirical CDF
        Z_i = emp_cdf.inverse(U_i) 
        Z.append(torch.Tensor(Z_i))

    Z = torch.stack(Z, axis=0).T
    return Z

def train_idx_sampler(tr_idx:int, context_len:int, prediction_len:int, num_samples:int) -> list:
    """
    Given a traini index (a point), with context length and prediction length, this function samples a sequence of length 
    context_len + prediction_len for 'num_samples' times.
    
    INPUTS
        tr_idx:int
            an end point of training dataset. It is an integer
        context_len:int
            specifies the length of training interval. 
        prediction_len:int
            specifies the length of prediction interval. 
        num_samples:
            specifies the number of samples to be sampled
    
    RETURNS
        sample_indices: list
            a list of sampled indices each in the form of (training indices, prediction indices)
    """
    sample_indices = []
    for idx in np.random.randint(0,tr_idx - context_len, size=num_samples):
        sample_indices.append((torch.LongTensor(idx + np.arange(context_len)), \
                            torch.LongTensor(idx + np.arange(prediction_len)+ context_len)))
    return sample_indices

In [38]:
class MultiVariateGP(nn.Module):
    def __init__(self, input_size:int, rank_size:int):
        super(MultiVariateGP,self).__init__()
        self.input_size = input_size
        self.rank_size  = rank_size

        # define linear weight for calculating parameters of gaussian process
        # these weights are SHARED across all time series.
        self.layer_m = nn.Linear(input_size, 1) # mean 
        self.layer_d = nn.Linear(input_size, rank_size) # diagonal point
        self.layer_v = nn.Sequential(
                                        nn.Linear(input_size, 1),
                                        nn.Softplus(beta=1)
                                    ) # volatility

    def forward(self, h_t:torch.Tensor):
        m_t = self.layer_m(h_t)
        d_t = self.layer_d(h_t)
        v_t = self.layer_v(h_t)
        return m_t, d_t, v_t


class GPAutoregressiveRNN(nn.Module):
    def __init__(
        self, 
        input_size:int, 
        hidden_size:int, 
        num_layers:int, 
        batch_size:int,
        num_asset:int,
        rank_size:int,
        dropout:float=0.1,
        batch_first:bool=False,
        features:Optional[dict]=None 
    ):
        super(GPAutoregressiveRNN,self).__init__()
        self.input_size   = input_size
        self.hidden_size  = hidden_size
        self.batch_size   = batch_size
        self.num_asset    = num_asset
        self.num_layers   = num_layers
        self.batch_first  = batch_first
        self.feature_size = len(features[0]) if features is not None else 0
        self.features = features
        self.hidden   = None
        
        # local lstm
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                             num_layers=num_layers, batch_first=batch_first, dropout=dropout)
        
        # Multivariate Gaussian Process
        self.gp = MultiVariateGP(input_size=hidden_size + self.feature_size, rank_size=rank_size)

    def init_weight(self) -> None:
        h0, c0 = torch.zeros(self.num_layers, self.num_asset, self.hidden_size),\
                    torch.zeros(self.num_layers, self.num_asset, self.hidden_size)
        self.hidden = (h0, c0)

    def feature_selector(self, indices:torch.Tensor, time_steps:int) -> torch.Tensor:
        feature = []
        for idx in indices:
            feature.append(self.features[idx.item()].repeat(time_steps,1))
        feature = torch.stack(feature, axis=1)
        return feature

    def forward(self, z_t:torch.Tensor, pred:bool=False) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        assert len(z_t.size()) == 2, 'input tensor dimension is not equal to 2.'
        z_t = z_t.unsqueeze(2)

        # select the batch then pass it through the unrolled LSTM
        # timestep x num_batch x 1
        output, (hn, cn) = self.lstm(z_t, self.hidden)
        self.hidden = (hn.detach(), cn.detach())
        
        # we only use the subset of batch for training
        # else return all indices
        if pred:
            batch_indices = torch.arange(self.num_asset)
        else:
            batch_indices = torch.randperm(self.num_asset)[:self.batch_size]
        
        # find feature vector e_i for each asset i then concatenate it with LSTM output.
        e = self.feature_selector(batch_indices, z_t.size(0))
        y_t = torch.concat([output[:,batch_indices,:], e], axis=2)

        # get GP parameters
        # calculate parameters of multivariate gaussian process for each time t
        mu_t, v_t, d_t = self.gp(y_t)
        cov_t = torch.diag_embed(d_t.squeeze(2)) + (v_t @ v_t.permute(0,2,1)) # D + V @ V.T
        return mu_t, cov_t, batch_indices


In [39]:
prices = pd.read_csv(DATA_PATH+'/prices.csv', index_col='Date')

# preprocess weekly data
prices.index = pd.to_datetime(prices.index)
lret_d = np.log(prices/prices.shift(1)).fillna(0.0) # weekly return
prices = prices.resample('M').last()
lret_m = np.log(prices/prices.shift(1)).fillna(0.0) # weekly return

In [40]:
# define feature vector, which is speical to each time series
# first 5: stock, gov bond, corp bond, real estate, commodity
# last 3 : us, europe, other

e1 = torch.Tensor([1,0,0,0,0,0,1,0]) # IEV, europe
e2 = torch.Tensor([1,0,0,0,0,0,0,1]) # EEM, em. market
e3 = torch.Tensor([0,0,1,0,0,1,0,0]) # AGG, us corp bonds
e4 = torch.Tensor([0,1,0,0,0,1,0,0]) # IEF, us gov bonds
e5 = torch.Tensor([0,0,0,1,0,1,0,0]) # IYR, us real estates
e6 = torch.Tensor([0,0,0,0,1,0,0,0]) # IAU, gold
e7 = torch.Tensor([1,0,0,0,0,1,0,0]) # ITOT, us equities

e_dict = {0: e1, 1: e2, 2: e3, 3: e4, 4: e5, 5: e6, 6: e7}

In [13]:
#TODO run Optuna
#TODO add mean and covariance to existing RP strategy and do the plotting

In [263]:
# define the GP-Copula model and initialize the weight
model = GPAutoregressiveRNN(input_size=1, hidden_size=4, num_layers=2, rank_size=4, 
                             batch_size=3, num_asset=7, dropout=0.05, features=e_dict)
model.init_weight()

# set up the loss function and optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3, weight_decay= 1e-5)
num_epochs = 250

# convert the data to torch tensor
split_idx = int(lret_m.shape[0] * 0.7)
Z_tr, Z_te = torch.Tensor(lret_m.iloc[:split_idx].values), torch.Tensor(lret_m.iloc[split_idx:].values)

# transform 
X_tr, cdfs = transform(Z_tr)
X_te, _    = transform(Z_te, cdfs)


for epoch in range(num_epochs):
    # randomly sample sequence index for training
    sampler = train_idx_sampler(tr_idx=Z_tr.size(0), context_len=12, prediction_len=1, num_samples=50)
    losses = []
    losses_valid = torch.Tensor([0.0])

    for tr_idx, te_idx in sampler: # for each sequence sample
        optimizer.zero_grad()

        # run the model
        mu_t, cov_t, batch_idx = model(Z_tr[tr_idx])
        x = X_tr[tr_idx][:,batch_idx]

        # gaussian log-likelihood loss
        loss = gaussian_loss(x, mu_t, cov_t)
        loss.backward()
        optimizer.step()
        
        # append loss of each batch
        losses.append(loss.detach())

        # prediction step
        mu_pred, cov_pred, _ = model(Z_tr[te_idx], pred=True)
        Z_tr_hat = inv_transform(mu_pred.detach(), cdfs)

        losses_valid = losses_valid + (Z_tr_hat[0]- Z_tr[te_idx]).pow(2).sum()


    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}') 
        print(f'Gaussian LL Loss : {np.round(np.mean(losses),2)}')
        print(f'Validation MSE   : {losses_valid[0]/50} \n')


Epoch 1
Gaussian LL Loss : 46.08000183105469
Validation MSE   : 0.010186588391661644 

Epoch 11
Gaussian LL Loss : -3.559999942779541
Validation MSE   : 0.011457579210400581 

Epoch 21
Gaussian LL Loss : -45.540000915527344
Validation MSE   : 0.012191014364361763 

Epoch 31
Gaussian LL Loss : -57.04999923706055
Validation MSE   : 0.011357217095792294 

Epoch 41
Gaussian LL Loss : -49.209999084472656
Validation MSE   : 0.008757762610912323 

Epoch 51
Gaussian LL Loss : -63.650001525878906
Validation MSE   : 0.0055421944707632065 

Epoch 61
Gaussian LL Loss : -73.20999908447266
Validation MSE   : 0.01150986086577177 

Epoch 71
Gaussian LL Loss : -57.25
Validation MSE   : 0.009251376613974571 

Epoch 81
Gaussian LL Loss : -56.13999938964844
Validation MSE   : 0.015335332602262497 

Epoch 91
Gaussian LL Loss : -65.9800033569336
Validation MSE   : 0.012301173992455006 

Epoch 101
Gaussian LL Loss : -65.0999984741211
Validation MSE   : 0.012142322957515717 

Epoch 111
Gaussian LL Loss : -56.